In [1]:
from pyscf import gto,scf
import pyscf
#from alch_deriv import alch_deriv
import matplotlib.pyplot as plt
from scipy.interpolate import InterpolatedUnivariateSpline as spl
import numpy as np
from FcMole import FcM
#from aaff import aaff
ang2bohr=1.8897261246
bohr2ang=.5291772109

In [4]:
CO=gto.M(atom="C 0 0 0; O 0 0 2.05",unit='Bohrs',basis="ccpv5z",verbose=0)
hfCO=scf.RHF(CO)
CO.nao

182

In [5]:
def optimizer2at(a1,a2,fcs=None):
    d1=np.linspace(1.9,2.4,9)
    es=[]
    for d in d1:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpv5z",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpv5z",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(1.9,2.4,501)
    bspline=spl(d1,es)
    bspline(dspl)
    dminspline=1.9+.001*(bspline(dspl).argmin())
    d2=np.linspace(dminspline-.03,dminspline+.03,7)
    es=[]
    for d in d2:
        if fcs==None:
            mol=gto.M(atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpv5z",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf())
        else:
            mol=FcM(fcs=fcs,atom="{} 0 0 0; {} 0 0 {} ".format(a1,a2,d),unit='Bohrs',basis="ccpv5z",verbose=0)
            mf=scf.RHF(mol)
            es.append(mf.scf(dm0=mf.init_guess_by_1e()))
    dspl=np.linspace(dminspline-.03,dminspline+.03,6001)
    bspline=spl(d2,es)
    #plt.plot(dspl,bspline(dspl))
    COmin=dspl[bspline(dspl).argmin()]
    return COmin,bspline(dspl).min()

In [6]:
optimizer2at("C","O")

(2.08215, -112.79241173071513)

In [7]:
optimizer2at("N","N")

(2.01333, -108.99618801436145)

In [8]:
optimizer2at("B","F")

(2.3521099999999997, -124.16878064782895)

# GAUGE THE ERROR DUE TO BASIS SET

In [9]:
#CO@NN
optimizer2at("N","N",fcs=[-1,1.])

(2.0817099999999997, -112.78341348258633)

In [10]:
#CO@BF
optimizer2at("B","F",fcs=[1.,-1.])

(2.08053, -112.77984015681389)

In [11]:
#BF@CO
optimizer2at("C","O",fcs=[-1,1])

(2.35174, -124.16039049709357)

In [12]:
#NN@CO
optimizer2at("C","O",fcs=[1,-1])

(2.0123599999999997, -108.98642383980003)

In [1]:
import pandas as pd
q=[]

In [2]:
l={"basis set":"cc-pV5Z"
    ,"nAO":182
    ,"COm":2.08215
    ,"COe": -112.79241173071513
    ,"NNm":2.01333
    ,"NNe":-108.99618801436145
    ,"BFm":2.3521099999999997
    ,"BFe":-124.16878064782895
    ,"CO@NNm":2.0817099999999997
    ,"CO@NNe": -112.78341348258633
    ,"CO@BFm":2.08053
    ,"CO@BFe":-112.77984015681389
    ,"BF@COm":2.35174
    ,"BF@COe":-124.16039049709357
    ,"NN@COm":2.0123599999999997
    ,"NN@COe":-108.98642383980003
}
q.append(l)

In [3]:
df=pd.DataFrame(q)
df.to_pickle("cc-pV5Z")

In [8]:
df

,BF@COe,BF@COm,BFe,BFm,CO@BFe,CO@BFm,CO@NNe,CO@NNm,COe,COm,NN@COe,NN@COm,NNe,NNm,basis set,nAO
0,-124.16039,2.35174,-124.168781,2.35211,-112.77984,2.08053,-112.783413,2.08171,-112.792412,2.08215,-108.986424,2.01236,-108.996188,2.01333,cc-pV5Z,182


In [5]:
dft=pd.read_pickle("minima")

In [7]:
dft.append(df).to_pickle("minima_5z")